## CCTVストリーム画像から災害検知するエージェント
### 🧩 システム構成（MVP）
- graph TD
- A[CCTV画像ストリーム] --> B[画像キャプチャ & 前処理]
- B --> C[UI-TARS-1.5: Image-to-Text]
- C --> D[災害キーワード、災害スコア判定モジュール]
- D --> E[位置特定（カメラID → 地図）]
- E --> F[被害要約生成]
- F --> G[自然言語レポート生成]

#### 1. 📸 画像キャプチャ & 前処理
- 各CCTVから定期的に画像を取得（例：1分ごと）
- 解像度調整、ノイズ除去、タイムスタンプ付与

##### 🔧 必要なツール
- | ツール | 用途 | 
- | ffmpeg | ストリームからフレームを抽出 | 
- | OpenCV | Pythonで画像処理と保存 | 
##### インストール
- pip install yt-dlp opencv-python
- ⇒Successfully installed opencv-python-4.12.0.88 yt-dlp-2025.6.30 
- ▼Not well：sudo apt install ffmpeg
##### sudo無効の仮想環境での対処法
- そのメッセージは、お使いの環境（一部の仮想環境）で sudo 権限が無効化されていることを示しています。以下に、状況別の対処法を整理しました：
- 🛠️ 対処法：sudo が無効な環境で ffmpeg を使うには？
- ✅ 方法①：ffmpeg をバイナリで直接インストール（推奨）
- 公式ffmpegビルドサイト または FFmpeg GitHub から静的バイナリをダウンロード、例（Linux x86_64）:
- 実行ファイルをパスに追加（例：~/.local/bin）
- 動作確認
##### ffmpeg.exe Windows用
- ffmpeg（拡張子なし）は Linux用の実行バイナリ
- Windowsでは .exe 拡張子のある Windowsネイティブビルド が必要
✅ 解決策：Windows向けの正しい ffmpeg.exe を入手する
以下の手順で、Windows用の ffmpeg.exe を確実に取得できます：
🔗 ステップ1：公式Windowsビルドサイトにアクセス
👉 https://www.gyan.dev/ffmpeg/builds/
📦 ステップ2：次のいずれかをダウンロード
- ffmpeg-release-essentials.zip（軽量で必要最小限）、解凍し、exeを確認
✅フォルダーを作成
- New-Item -ItemType Directory -Path "C:\Tools\ffmpeg\bin" -Force
✅環境変数 PATH に追加（永続化）
- $env:Path += ";C:\Tools\ffmpeg\bin"
✅ 動作確認
- ffmpeg -version → バージョン情報が表示されれば成功です！
- ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers

### 🎥 ストリーム画像取得スクリプト（Windows環境向け）
- CCTVストリームからの画像取得スクリプトの整備に進みましょう。
- 関東地方整備局より、ライブ映像が試験配信されている。
- 10秒ごとに、カメラが切り替わっている。
- 次の「河川YouTubeライブ映像」を対象に、10秒ごとに静止画像を保存できるMVPコードを用意します。
- 【試験配信】関東地方整備局　利根川・烏川　ライブ映像

✅ 必要なツール
| ツール | 用途 | 
| yt-dlp | YouTubeライブのストリームURLを取得 | 
| ffmpeg.exe | ストリームから静止画を抽出 | 
| PowerShell | スケジュール＆保存処理 | 

- PowerShellスクリプトをPythonから実行できるように修正しました。
新しいCell In[1]は、subprocessを使ってPowerShellスクリプトを呼び出すPythonコードになっています。
- このまま実行すれば、PowerShellの処理がPythonから開始されます。

In [2]:
import subprocess
powershell_script = r'''
$youtubeURL = "https://www.youtube.com/live/IyE0H7nKP68?si=vaTbIU5cE8VFtQeg"  # 河川YouTubeライブ映像のURL
$savePath   = "C:\Users\yasun\PyTorch\CCTVDisasterAgent\ToneKarasuFrames"
$ffmpegPath = "C:\Tools\ffmpeg\bin\ffmpeg.exe"  # 必要に応じて変更
$interval   = 10  # 10秒
$Num_of_Captures = 20 # 画像取得回数, 200秒分
New-Item -ItemType Directory -Path $savePath -Force | Out-Null
for ($i = 0; $i -lt $Num_of_Captures; $i++) {
    $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
    $imageFile = "$savePath\frame_$timestamp.jpg"
    $streamURL = yt-dlp -g $youtubeURL
    & "$ffmpegPath" -y -i "$streamURL" -frames:v 1 "$imageFile"
    Write-Host "✅ Saved: $imageFile"
    Start-Sleep -Seconds $interval
}
'''
subprocess.run(['powershell', '-Command', powershell_script])

CompletedProcess(args=['powershell', '-Command', '\n$youtubeURL = "https://www.youtube.com/live/IyE0H7nKP68?si=vaTbIU5cE8VFtQeg"  # 河川YouTubeライブ映像のURL\n$savePath   = "C:\\Users\\yasun\\PyTorch\\CCTVDisasterAgent\\ToneKarasuFrames"\n$ffmpegPath = "C:\\Tools\\ffmpeg\\bin\\ffmpeg.exe"  # 必要に応じて変更\n$interval   = 10  # 10秒\n$Num_of_Captures = 20 # 画像取得回数, 200秒分\nNew-Item -ItemType Directory -Path $savePath -Force | Out-Null\nfor ($i = 0; $i -lt $Num_of_Captures; $i++) {\n    $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"\n    $imageFile = "$savePath\\frame_$timestamp.jpg"\n    $streamURL = yt-dlp -g $youtubeURL\n    & "$ffmpegPath" -y -i "$streamURL" -frames:v 1 "$imageFile"\n    Write-Host "✅ Saved: $imageFile"\n    Start-Sleep -Seconds $interval\n}\n'], returncode=0)

### 2. 🧠 Claudeによる画像理解(Image-to-Text LLM Prediction)
- 各画像に対して以下のプロンプトを付与：
- <|system|> あなたは災害監視エージェントです。
- <|user|> この画像に洪水の兆候がありますか？水位、道路冠水、土砂崩れなどを説明してください。
- <|vision_start|>...画像トークン...<|vision_end|>
##### ⇒出力：自然言語による被害説明（例：「道路が冠水しており、車両が立ち往生している」）

#### Anthropic APIの anthropic-version ヘッダー
- "2023-06-01" または "2023-10-01" など、公式ドキュメントで指定された有効なバージョンのみが使えます。

In [1]:
# Claude モデルを使用するための準備
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

# 1. 画像をbase64エンコード
import base64

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# 2. Claude APIリクエスト構築（被害説明プロンプト）
import requests

image_path = r"C:\Users\yasun\PyTorch\CCTVDisasterAgent\5_CaseKanto\KantoYT1_20250714_034402.jpg"

image_b64 = encode_image(image_path)  # 画像パスを適宜変更

payload = {
  "model": "claude-3-5-sonnet-20240620",
  "messages": [{
    "role": "user",
    "content": [
      {"type": "text", "text": "Please analyze this image and describe any signs of flooding, road damage, or disaster impact."},
      {
        "type": "image",
        "source": {
          "type": "base64",
          "media_type": "image/jpeg",
          "data": image_b64
        }
      }
    ]
  }],
  "max_tokens": 2048 #1024
}

response = requests.post(
    "https://api.anthropic.com/v1/messages",
    headers={
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    },
    json=payload
)

print(response.json()["content"][0]["text"])

This image shows a view of the Tone River (利根川) in Chiba Prefecture, Japan. The image is a night view, likely from a monitoring camera, as indicated by the text overlay providing location details.

Regarding signs of flooding, road damage, or disaster impact:

1. Water level: The river appears to be at a normal level, not showing any obvious signs of flooding. The banks of the river are clearly visible and not submerged.

2. Riverbank condition: The grassy area along the riverbank seems intact and not eroded, which would be a sign if there had been recent flooding.

3. Infrastructure: There are no visible signs of damage to any structures or roads in the image. The paths visible on the riverbank appear to be in normal condition.

4. Debris: There's no evident debris or displaced objects that might indicate recent flooding or storm damage.

5. Lighting: The city lights in the background are functioning, suggesting no widespread power outages that might occur during a disaster.

Overall,

In [2]:
# Claude APIレスポンスのデバッグ表示
print('Status Code:', response.status_code)
print('Response JSON:', response.json())

Status Code: 200
Response JSON: {'id': 'msg_01C4b9gFx8Z4eP2T9QCRbxWb', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20240620', 'content': [{'type': 'text', 'text': "This image shows a view of the Tone River (利根川) in Chiba Prefecture, Japan. The image is a night view, likely from a monitoring camera, as indicated by the text overlay providing location details.\n\nRegarding signs of flooding, road damage, or disaster impact:\n\n1. Water level: The river appears to be at a normal level, not showing any obvious signs of flooding. The banks of the river are clearly visible and not submerged.\n\n2. Riverbank condition: The grassy area along the riverbank seems intact and not eroded, which would be a sign if there had been recent flooding.\n\n3. Infrastructure: There are no visible signs of damage to any structures or roads in the image. The paths visible on the riverbank appear to be in normal condition.\n\n4. Debris: There's no evident debris or displaced objects that

### 3. 🚨 災害キーワード抽出、災害スコア判定モジュール
- 出力文から災害関連キーワードを抽出（例：冠水、氾濫、土砂）
- スコアリングして「災害あり／なし」を判定

In [3]:
def score_flood_claude(description: str) -> float:
    keywords = {
        "flood": 0.4,
        "overflow": 0.3,
        "impassable": 0.4,
        "damage": 0.3,
        "debris": 0.2,
        "storm": 0.2,
        "high water": 0.3,
        "water level": 0.3,
        "bridge failure": 0.5,
        "road washed out": 0.5
    }

    score = 0.0
    lowered = description.lower()

    # キーワードごとにスコアを加算
    for word, weight in keywords.items():
        if word in lowered:
            score += weight
    return min(score, 3.0)

# 該当したキーワード抽出してリスト化
def extract_keywords(description: str) -> list:
    keywords = {
        "flood": 0.4,
        "overflow": 0.3,
        "impassable": 0.4,
        "damage": 0.3,
        "debris": 0.2,
        "storm": 0.2,
        "high water": 0.3,
        "water level": 0.3,
        "bridge failure": 0.5,
        "road washed out": 0.5
    }

    found_keywords = []
    lowered = description.lower()

    for word in keywords.keys():
        if word in lowered:
            found_keywords.append(word)

    return found_keywords

# 被害説明の抽出とスコア計算、キーワードリストを表示
description_text = response.json()["content"][0]["text"]
print("📜 被害説明:", description_text)

score = score_flood_claude(description_text)
print(f"📊 災害スコア: {score:.2f}")
found_disaster_words = extract_keywords(description_text)
print("🔍 該当キーワード:", found_disaster_words)

📜 被害説明: This image shows a view of the Tone River (利根川) in Chiba Prefecture, Japan. The image is a night view, likely from a monitoring camera, as indicated by the text overlay providing location details.

Regarding signs of flooding, road damage, or disaster impact:

1. Water level: The river appears to be at a normal level, not showing any obvious signs of flooding. The banks of the river are clearly visible and not submerged.

2. Riverbank condition: The grassy area along the riverbank seems intact and not eroded, which would be a sign if there had been recent flooding.

3. Infrastructure: There are no visible signs of damage to any structures or roads in the image. The paths visible on the riverbank appear to be in normal condition.

4. Debris: There's no evident debris or displaced objects that might indicate recent flooding or storm damage.

5. Lighting: The city lights in the background are functioning, suggesting no widespread power outages that might occur during a disaster.



#### JSON形式で出力

In [4]:
# JSON形式でのレスポンスを保存
import json

# found_disaster_wordsをテキスト化
import os
found_disaster_words_text = ", ".join(found_disaster_words) if found_disaster_words else "None"

response_data = {
    "description": description_text,
    "score": score,
    "found_disaster_words": found_disaster_words_text
}   

#### 5. 📝 被害要約生成
- 被害内容を要約し、以下のようなレポートを生成：
- 【災害報告】
- 主な災害事象：キーワード
- 災害スコア：1.3
- 災害を検知した画像：ファイル名
- 検知時刻：2025年7月9日 06:15
- 状況：道路が冠水し、車両が通行不能。水位は歩道を越えている。
- 詳細な被害説明：画像から推論出力の全文

In [5]:
from datetime import datetime

def generate_disaster_report(description, keywords, score, image_filename, detected_time,description_text):
    # スコアの正規化
    normalized_score = min(score, 1.0)

    # フォーマット用の時刻変換（例: "20250709_0615" → "2025年7月9日 06:15"）
    def format_timestamp(ts_str):
        try:
            dt = datetime.strptime(ts_str, "%Y%m%d_%H%M")
            return dt.strftime("%Y年%m月%d日 %H:%M")
        except:
            return detected_time  # 変換できなければそのまま使う

    # レポート本文の生成
    report = f"""
【災害報告】
- 主な災害事象：{keywords}
- 災害スコア：{normalized_score:.1f}
- 災害を検知した画像：{image_filename}
- 検知時刻：{format_timestamp(image_filename.split("frame_")[-1].split(".jpg")[0])}
- 状況：{description}
- 詳細な推論と説明：{description_text}
""".strip()

    return report

# レポート生成＆表示
report_text = generate_disaster_report(
    description=response_data["description"],
    keywords=response_data["found_disaster_words"],
    score=response_data["score"],
    image_filename=os.path.basename(image_path),
    detected_time="2025年7月9日 06:15",
    description_text=description_text
)

print(report_text)

#　report_textをJSON形式で出力
import json
report_json = {
    "report": report_text,
    "image_filename": os.path.basename(image_path),
    "detected_time": "2025年7月9日 06:15"
}
#   保存するファイル名に、画像ファイル名を含める
report_filename = f"disaster_report_{os.path.basename(image_path).split('.')[0]}.json"
with open(report_filename, "w", encoding="utf-8") as f:
    json.dump(report_json, f, ensure_ascii=False, indent=4)
print(f"📝 レポートを保存しました: {report_filename}")


【災害報告】
- 主な災害事象：flood, damage, debris, storm, water level
- 災害スコア：1.0
- 災害を検知した画像：KantoYT1_20250714_034402.jpg
- 検知時刻：2025年7月9日 06:15
- 状況：This image shows a view of the Tone River (利根川) in Chiba Prefecture, Japan. The image is a night view, likely from a monitoring camera, as indicated by the text overlay providing location details.

Regarding signs of flooding, road damage, or disaster impact:

1. Water level: The river appears to be at a normal level, not showing any obvious signs of flooding. The banks of the river are clearly visible and not submerged.

2. Riverbank condition: The grassy area along the riverbank seems intact and not eroded, which would be a sign if there had been recent flooding.

3. Infrastructure: There are no visible signs of damage to any structures or roads in the image. The paths visible on the riverbank appear to be in normal condition.

4. Debris: There's no evident debris or displaced objects that might indicate recent flooding or storm damage.

5. Lighti

#### (PEND) 📍 発生位置の特定
- 災害あり判定のカメラIDを特定
- カメラ位置のGPS座標のマッピングテーブルを用意
- (PEND)発生地点を地図上にプロット（例：Plotly, Leaflet）